In [ ]:
from wq_api import *

session = sign_in(username,password)

In [ ]:
# data is restored in {index}_refine_1.csv, where index is generated by order
import os

dir_path = './refine_data'

if not os.path.exists(dir_path):
    os.makedirs(dir_path)

index = 0
while True:
    to_path = f'{dir_path}/{index}_refine_1.csv'
    if not os.path.exists(to_path):
        break
    index += 1

print(f'Next available file path: {to_path}')

In [ ]:
setting = {
    'universe':['TOP3000', 'MINVOL1M'],
    'decay':[4,6,10,20,25,30,60,120,252],
    'neutralization':['MARKET','INDUSTRY','SUBINDUSTRY','SECTOR','COUNTRY']
}

In [ ]:
# make alpha template
alphas = []

alpha = "market_return = group_mean(returns,1,market);fear = ts_mean(abs(returns - market_return)/(abs(returns)+abs(market_return)),20);vhat = ts_regression(volume,ts_mean(anl15_s_12_m_mean,60),120);ehat = ts_regression(returns-market_return,vhat,120);alpha = group_neutralize(-ehat*rank(fear),bucket(rank(cap),range='0,1,0.1'));final = group_zscore(alpha,densify(country));is_nan(final)?0.0001:final"
alphas.append(alpha)

In [ ]:
# make test alpha list
priority_alpha_list=[]
priority_alpha_list = [
    {
        'type': 'REGULAR',
        'settings': {
            'instrumentType': 'EQUITY',
            'region': 'GLB',
            'universe': universe,
            'delay': 1,
            'decay': decay,
            'neutralization': neutralization,
            'truncation': 0.08,
            'pasteurization': 'ON',
            'unitHandling': 'VERIFY',
            'nanHandling': 'ON',
            'language': 'FASTEXPR',
            'visualization': False,
        },
        'regular': alpha
    }
    for alpha in alphas
    for universe in setting['universe']
    for decay in setting['decay']
    for neutralization in setting['neutralization']
]
    # simulation_response = session.post('https://api.worldquantbrain.com/simulations', json=simulation_data)

In [ ]:
import pandas as pd
# 将 priority_alpha_list 转换为 DataFrame
priority_alpha_df = pd.DataFrame(priority_alpha_list)

# 将 settings 列中的字典展开为单独的列
settings_df = priority_alpha_df['settings'].apply(pd.Series)

# 将展开的 settings 列与其他列合并
priority_alpha_df = pd.concat([priority_alpha_df.drop(columns=['settings']), settings_df], axis=1)

print(priority_alpha_df)

priority_alpha_df.to_csv(to_path)